<a href="https://colab.research.google.com/github/Pushkar0655g/Generative-AI/blob/main/40enhanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. OpenAI Whisper

In [1]:
# Install dependencies
!pip install -q git+https://github.com/openai/whisper.git  # Install Whisper
!pip install -q yt-dlp  # Install yt-dlp for YouTube downloading
!pip install -q pydrive  # Install PyDrive for Google Drive integration
!apt-get install -y ffmpeg  # Ensure FFmpeg is installed

# Import all libraries
from google.colab import files, auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import whisper
import subprocess
import io
import os

# Define paths
video_path = "/content/video.mp4"  # Temporary path for the video
srt_path = "/content/subtitles.srt"  # SRT file path

# Function to download YouTube video
def download_youtube_video(url, output_path):
    try:
        command = f"yt-dlp -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4' -o {output_path} {url}"
        subprocess.run(command, shell=True, check=True)
        print(f"Downloaded video to {output_path}")
        return True
    except subprocess.CalledProcessError as e:
        raise RuntimeError(f"Failed to download video from {url}: {e}")

# Function to authenticate and access Google Drive
def authenticate_drive():
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleDrive(gauth).auth.credentials
    drive = GoogleDrive(gauth)
    return drive

# Function to download video from Google Drive
def download_drive_video(file_id, output_path):
    try:
        drive = authenticate_drive()
        file = drive.CreateFile({'id': file_id})
        file.GetContentFile(output_path)
        print(f"Downloaded video from Google Drive to {output_path}")
        return True
    except Exception as e:
        print(f"Error downloading from Google Drive: {str(e)}")
        return False

# Load Whisper model
model = whisper.load_model("base")  # "base" for speed, "large" for better accuracy

# Function to generate SRT file from transcription
def create_srt(segments, filename):
    with open(filename, "w") as f:
        for i, segment in enumerate(segments, 1):
            start = f"{segment['start']:.3f}".replace(".", ",")
            end = f"{segment['end']:.3f}".replace(".", ",")
            text = segment["text"].strip()
            f.write(f"{i}\n00:00:{start} --> 00:00:{end}\n{text}\n\n")
    print(f"Subtitles saved to {filename}")

# Function to process video and generate subtitles
def process_video(video_path, srt_path):
    try:
        # Transcribe the video
        print("Processing video...")
        result = model.transcribe(video_path, language="en")  # Transcribe English audio to English subtitles
        create_srt(result["segments"], srt_path)
        return True
    except Exception as e:
        print(f"Error processing video: {str(e)}")
        return False

# Main function to handle user input and process video
def main():
    print("Choose the source of the video:")
    print("1. YouTube")
    print("2. Google Drive")

    choice = input("Enter your choice (1 or 2): ").strip()

    if choice == "1":
        youtube_link = input("Enter the YouTube video URL: ").strip()
        if not youtube_link:
            print("Error: YouTube URL cannot be empty.")
            return
        success = download_youtube_video(youtube_link, video_path)
        if not success:
            print("Failed to download YouTube video. Please check the URL and try again.")
            return

    elif choice == "2":
        print("To download from Google Drive, ensure the video file is shared publicly or accessible to your Google account.")
        drive_link = input("Enter the Google Drive file ID (from the shareable link): ").strip()
        if not drive_link:
            print("Error: Google Drive file ID cannot be empty.")
            return
        success = download_drive_video(drive_link, video_path)
        if not success:
            print("Failed to download Google Drive video. Please check the file ID and ensure it is accessible.")
            return

    else:
        print("Invalid choice. Please select 1 for YouTube or 2 for Google Drive.")
        return

    # Process the video to generate subtitles
    print("Starting video processing...")
    success = process_video(video_path, srt_path)

    # Display and download subtitles if successful
    if success:
        print("\nSample of subtitles:")
        with open(srt_path, "r") as f:
            print(f.read(500))  # Show first 500 characters
        files.download(srt_path)
    else:
        print("Video failed to process. Please try again.")

# Run the main function
if __name__ == "__main__":
    main()

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.2 MB/s eta 0

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 82.2MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Choose the source of the video:
1. YouTube
2. Google Drive
Enter your choice (1 or 2): 1
Enter the YouTube video URL: https://youtu.be/TephzFnZCpA?si=yx4pIxDctMmCwahy
Downloaded video to /content/video.mp4
Starting video processing...
Processing video...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Subtitles saved to /content/subtitles.srt

Sample of subtitles:
1
00:00:0,000 --> 00:00:7,000
Hi guys, welcome back to my channel. If you haven't seen my previous video, the delivery of the Bugatti Bulli at the dealer.

2
00:00:7,000 --> 00:00:12,000
Go check it out, but today's a special day because it's actually the Bulli is coming home.

3
00:00:12,000 --> 00:00:14,000
And there it is.

4
00:00:14,000 --> 00:00:17,000
How are you? Good to see you again.

5
00:00:17,000 --> 00:00:18,000
Very nice.

6
00:00:18,000 --> 00:00:21,000
Not only we're getting in 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 MarianMT (Hugging Face)

In [2]:
# Install dependencies
!pip install -q git+https://github.com/openai/whisper.git  # Install Whisper
!pip install -q yt-dlp  # Install yt-dlp for YouTube downloading
!pip install -q transformers  # Install transformers for Hindi translation
!pip install -q pydrive  # Install PyDrive for Google Drive integration
!apt-get install -y ffmpeg  # Ensure FFmpeg is installed

# Import all libraries
from google.colab import files, auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import whisper
import subprocess
from transformers import MarianMTModel, MarianTokenizer
import io
import os

# Define paths
video_path = "/content/video.mp4"  # Temporary path for the video
srt_path = "/content/subtitles_hindi.srt"  # SRT file path (named for clarity)

# Function to download YouTube video
def download_youtube_video(url, output_path):
    try:
        command = f"yt-dlp -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4' -o {output_path} {url}"
        subprocess.run(command, shell=True, check=True)
        print(f"Downloaded video to {output_path}")
        return True
    except subprocess.CalledProcessError as e:
        raise RuntimeError(f"Failed to download video from {url}: {e}")

# Function to authenticate and access Google Drive
def authenticate_drive():
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleDrive(gauth).auth.credentials
    drive = GoogleDrive(gauth)
    return drive

# Function to download video from Google Drive
def download_drive_video(file_id, output_path):
    try:
        drive = authenticate_drive()
        file = drive.CreateFile({'id': file_id})
        file.GetContentFile(output_path)
        print(f"Downloaded video from Google Drive to {output_path}")
        return True
    except Exception as e:
        print(f"Error downloading from Google Drive: {str(e)}")
        return False

# Load Whisper model
model = whisper.load_model("base")  # "base" for speed, "large" for better accuracy

# Load MarianMT model for English-to-Hindi translation
model_name = "Helsinki-NLP/opus-mt-en-hi"  # English to Hindi translation model
tokenizer = MarianTokenizer.from_pretrained(model_name)
translation_model = MarianMTModel.from_pretrained(model_name)

# Function to generate SRT file from transcription
def create_srt(segments, filename):
    with open(filename, "w") as f:
        for i, segment in enumerate(segments, 1):
            start = f"{segment['start']:.3f}".replace(".", ",")
            end = f"{segment['end']:.3f}".replace(".", ",")
            text = segment["text"].strip()
            f.write(f"{i}\n00:00:{start} --> 00:00:{end}\n{text}\n\n")
    print(f"Subtitles saved to {filename}")

# Function to process video with transcription and translation
def process_video(video_path, srt_path):
    try:
        # Transcribe the video to English
        print("Transcribing video to English...")
        result = model.transcribe(video_path, language="en")  # Transcribe English audio

        # Translate English segments to Hindi
        print("Translating English to Hindi...")
        hindi_segments = []
        for segment in result["segments"]:
            # Prepare English text for translation
            inputs = tokenizer(segment["text"], return_tensors="pt", padding=True)
            # Generate Hindi translation
            translated = translation_model.generate(**inputs)
            # Decode and update segment text to Hindi
            segment["text"] = tokenizer.decode(translated[0], skip_special_tokens=True)
            hindi_segments.append(segment)

        # Generate SRT with Hindi subtitles
        create_srt(hindi_segments, srt_path)
        return True
    except Exception as e:
        print(f"Error processing video: {str(e)}")
        return False

# Main function to handle user input and process video
def main():
    print("Choose the source of the video:")
    print("1. YouTube")
    print("2. Google Drive")

    choice = input("Enter your choice (1 or 2): ").strip()

    if choice == "1":
        youtube_link = input("Enter the YouTube video URL: ").strip()
        if not youtube_link:
            print("Error: YouTube URL cannot be empty.")
            return
        success = download_youtube_video(youtube_link, video_path)
        if not success:
            print("Failed to download YouTube video. Please check the URL and try again.")
            return

    elif choice == "2":
        print("To download from Google Drive, ensure the video file is shared publicly or accessible to your Google account.")
        drive_link = input("Enter the Google Drive file ID (from the shareable link): ").strip()
        if not drive_link:
            print("Error: Google Drive file ID cannot be empty.")
            return
        success = download_drive_video(drive_link, video_path)
        if not success:
            print("Failed to download Google Drive video. Please check the file ID and ensure it is accessible.")
            return

    else:
        print("Invalid choice. Please select 1 for YouTube or 2 for Google Drive.")
        return

    # Process the video to generate subtitles
    print("Starting video processing...")
    success = process_video(video_path, srt_path)

    # Display and download subtitles if successful
    if success:
        print("\nSample of Hindi subtitles:")
        with open(srt_path, "r") as f:
            print(f.read(500))  # Show first 500 characters
        files.download(srt_path)
    else:
        print("Video failed to process. Please try again.")

# Run the main function
if __name__ == "__main__":
    main()

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

...

Choose the source of the video:
1. YouTube
2. Google Drive
Enter your choice (1 or 2): 1
Enter the YouTube video URL: https://youtu.be/TephzFnZCpA?si=MkKAVgiYtnLStING
Downloaded video to /content/video.mp4
Starting video processing...
Transcribing video to English...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Translating English to Hindi...
Subtitles saved to /content/subtitles_hindi.srt

Sample of Hindi subtitles:
1
00:00:0,000 --> 00:00:7,000
हाय दोस्तों, मेरे चैनल के लिए वापस स्वागत है. यदि तुम मेरा पिछले वीडियो नहीं देखा है, सौदाक में बगीटी के प्रसव.

2
00:00:7,000 --> 00:00:12,000
जाओ जाँच करो, लेकिन आज एक विशेष दिन है क्योंकि यह वास्तव में बुली घर आ रहा है.

3
00:00:12,000 --> 00:00:14,000
और वहाँ यह है.

4
00:00:14,000 --> 00:00:17,000
आप कैसे हैं?

5
00:00:17,000 --> 00:00:18,000
बहुत अच्छा.

6
00:00:18,000 --> 00:00:21,000
न केवल हम कार में जा रहे हैं, हम कार के साथ कुछ माल मिल रहे हैं.

7
00:00:2


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>